In [ ]:
pip install scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [ ]:
import numpy as np
import pandas as pd


In [ ]:
movie = pd.read_csv('/content/movie.csv')
rating = pd.read_csv('/content/rating_df.csv')
genome_scores = pd.read_csv('/content/genome_scores.csv')
link = pd.read_csv('/content/link.csv')
tag = pd.read_csv('/content/tag.csv')
test_data = pd.read_csv('/content/test_v2.csv')

NameError: ignored

In [ ]:
# select the column 'movieId' from the test data
movieId = test_data['movieId']

# filter down the data to include only the information relevant to the movies in the test data
movie = movie[movie.movieId.isin(movieId)]
rating = rating[rating.movieId.isin(movieId)]
genome_scores = genome_scores[genome_scores.movieId.isin(movieId)]
tag = tag[tag.movieId.isin(movieId)]
link = link[link.movieId.isin(movieId)]


In [ ]:
train_data = movie.merge(rating, how = "left", on = "movieId" )
train_data.head()

,movieId,title,genres,userId,rating,timestamp
0,356,Forrest Gump (1994),Comedy|Drama|Romance|War,107,4.5,2014-04-12 21:27:49
1,356,Forrest Gump (1994),Comedy|Drama|Romance|War,108,3.0,1996-07-05 12:26:24
2,356,Forrest Gump (1994),Comedy|Drama|Romance|War,109,4.0,2003-07-10 17:48:13
3,356,Forrest Gump (1994),Comedy|Drama|Romance|War,110,5.0,2007-02-10 04:02:30
4,356,Forrest Gump (1994),Comedy|Drama|Romance|War,116,4.0,2005-11-23 02:06:25


In [ ]:
# Create a Reader object to specify the rating scale
reader = Reader(rating_scale=(1, 5))

# Load the data into a Surprise Dataset
train_dataset = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

# Specify the desired test set size
test_size = 12340

# Split the data into training and testing sets
Trainset, Testset = train_test_split(train_dataset, test_size=test_size, random_state=42)


In [ ]:
# SVD Model Training
svd_model = SVD()
svd_model.fit(Trainset)
predictions = svd_model.test(Testset)

In [ ]:
 # Calculate RMSE and MAE
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


RMSE: 0.9282
MAE:  0.7174


In [ ]:
pip install scikit-learn matplotlib

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Convert predictions to binary labels based on a threshold
threshold = 4.0
binary_labels = [1 if pred.est >= threshold else 0 for pred in predictions]

# Get the true labels from Testset
true_labels = [int(test_tuple[2]) for test_tuple in Testset]


# Calculate precision, recall, and f1_score
precision = precision_score(true_labels, binary_labels, average='weighted')
recall = recall_score(true_labels, binary_labels, average='weighted')
f1 = f1_score(true_labels, binary_labels, average='weighted')

# Print the metrics
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.000335837297179915
Recall: 0.016693679092382497
F1 Score: 0.0006579898700990305


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Concatenate 'userId' and 'movieId' columns and create a new column 'ID'
test_data['userId'] = test_data['userId'].astype(int)
test_data['ID'] = test_data['userId'].astype(str) + '_' + test_data['movieId'].astype(str)
#print(test_data.head())

# # Use the trained SVD model for predictions
test_data['prediction'] = test_data['ID'].apply(lambda x: svd_model.predict(x.split('_')[0], x.split('_')[1]).est)

# Select the desired columns
result_df = test_data[['ID', 'prediction']]

# Rename columns to 'ID' and 'rating'
result_df.columns = ['ID', 'rating']

result_df.to_csv('/content/predictions.csv', index=False)



NameError: ignored

In [ ]:
from google.colab import files

# download the csv file
files.download('/content/predictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>